<a href="https://colab.research.google.com/github/smoreira/TensorFlow2/blob/master/style_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Transferência de estilo neural

Este tutorial usa aprendizado profundo para compor uma imagem no estilo de outra imagem (você já desejou pintar como Picasso ou Van Gogh?). Isso é conhecido como * transferência de estilo neural * e a técnica é descrita em <a href="https://arxiv.org/abs/1508.06576" class="external"> Um algoritmo neural de estilo artístico </a> (Gatys et al.).

Nota: Este tutorial demonstra o algoritmo original de transferência de estilo. Otimiza o conteúdo da imagem para um estilo específico. As abordagens modernas treinam um modelo para gerar a imagem estilizada diretamente (semelhante a [cyclegan] (cyclegan.ipynb)). Essa abordagem é muito mais rápida (até 1000x). Um [módulo de estilização arbitrária de imagens] pré-treinado (https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/tf2_arbitrary_image_stylization.ipynb) está disponível no [TensorFlow Hub] (https: / /tensorflow.org/hub) e para o [TensorFlow Lite] (https://www.tensorflow.org/lite/models/style_transfer/overview).

A transferência de estilo neural é uma técnica de otimização usada para capturar duas imagens - uma imagem * content * e uma * imagem de referência * style (como uma obra de arte de um pintor famoso) - e misturá-las para que a imagem de saída se pareça com a imagem de conteúdo, mas "pintado" no estilo da imagem de referência de estilo.

Isso é implementado otimizando a imagem de saída para corresponder às estatísticas de conteúdo da imagem e às estatísticas de estilo da imagem de referência de estilo. Essas estatísticas são extraídas das imagens usando uma rede convolucional.


Por exemplo, vamos tirar uma foto desse cachorro e da composição 7 de Wassily Kandinsky:

<img src = "https://storage.googleapis.com/download.tensorflow.org/example_images/YellowLabradorLooking_new.jpg" width = "500px" />

[Yellow Labrador Looking] (https://commons.wikimedia.org/wiki/File:YellowLabradorLooking_new.jpg), do Wikimedia Commons

<img src = "https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg" width = "500px" />


Agora, como seria se Kandinsky decidisse pintar a imagem deste cão exclusivamente com esse estilo? Algo assim?

<img src = "https://tensorflow.org/tutorials/generative/images/stylized-image.png" style = "width: 500px;" />

## Setup


### Importar e configurar os módulos

In [0]:
import tensorflow as tf

In [0]:
import IPython.display as display

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12,12)
mpl.rcParams['axes.grid'] = False

import numpy as np
import PIL.Image
import time
import functools

In [0]:
def tensor_to_image(tensor):
  tensor = tensor*255
  tensor = np.array(tensor, dtype=np.uint8)
  if np.ndim(tensor)>3:
    assert tensor.shape[0] == 1
    tensor = tensor[0]
  return PIL.Image.fromarray(tensor)

Faça o download de imagens e escolha uma imagem de estilo e uma imagem de conteúdo:

In [0]:
content_path = tf.keras.utils.get_file('YellowLabradorLooking_new.jpg', 'https://storage.googleapis.com/download.tensorflow.org/example_images/YellowLabradorLooking_new.jpg')

style_path = tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg')

## Visualize a entrada

Defina uma função para carregar uma imagem e limite sua dimensão máxima a 512 pixels.

In [0]:
def load_img(path_to_img):
  max_dim = 512
  img = tf.io.read_file(path_to_img)
  img = tf.image.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.float32)

  shape = tf.cast(tf.shape(img)[:-1], tf.float32)
  long_dim = max(shape)
  scale = max_dim / long_dim

  new_shape = tf.cast(shape * scale, tf.int32)

  img = tf.image.resize(img, new_shape)
  img = img[tf.newaxis, :]
  return img

Crie uma função simples para exibir uma imagem:

In [0]:
def imshow(image, title=None):
  if len(image.shape) > 3:
    image = tf.squeeze(image, axis=0)

  plt.imshow(image)
  if title:
    plt.title(title)

In [0]:
content_image = load_img(content_path)
style_image = load_img(style_path)

plt.subplot(1, 2, 1)
imshow(content_image, 'Imagem de Entrada')

plt.subplot(1, 2, 2)
imshow(style_image, 'Estilo')

## Transferência rápida de estilo usando o TF-Hub

Este tutorial demonstra o algoritmo original de transferência de estilo, que otimiza o conteúdo da imagem para um estilo específico. Antes de entrar em detalhes, vamos ver como o módulo [TensorFlow Hub] (https://tensorflow.org/hub) faz:

In [0]:
import tensorflow_hub as hub
hub_module = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/1')
stylized_image = hub_module(tf.constant(content_image), tf.constant(style_image))[0]
tensor_to_image(stylized_image)